DIY Covid-19 Dashboard

The chart compares the number of first-time infections with reinfections and shows the changes in the total number of epidemic cases.

Reinfections: Displays data on COVID-19 reinfections over time. It refers to the number of cases where individuals were infected with COVID-19 and then got infected agai.

First Episodes: Shows the number of first-time COVID-19 infections over time. This reflects the trend and fluctuation in new infection caes.

Cumulative Cases (Cases): Presents the cumulative number of confirmed cases since the start of the epidemic. This shows the overall case burden and the severity of the epidemic.

In [5]:
from uk_covid19 import Cov19API
import pandas as pd
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
%matplotlib inline
plt.rcParams['figure.dpi'] = 100
with open("timeclassification.json", "rt") as INFILE:
    data=json.load(INFILE)
datalist=data['data']
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
index=pd.date_range(startdate, enddate, freq='D')
timeclassificationdf=pd.DataFrame(index=index, columns=['reinfections', 'firstepisodes','cases'])

    ################################################################################   
for entry in datalist: 
    date=parse_date(entry['date'])
    for column in ['reinfections', 'firstepisodes','cases']:
        if pd.isna(timeclassificationdf.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else .0
            timeclassificationdf.loc[date, column]=value
            
timeclassificationdf.fillna(0.0, inplace=True) 
timeclassificationdf.to_pickle("timeclassificationdf.pkl")

In [9]:
def wrangle_data(rawdata):
    with open("timeclassification.json", "rt") as INFILE:
        data=json.load(INFILE)
    datalist=data['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    def parse_date(datestring):
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    timeclassification=pd.DataFrame(index=index, columns=['reinfections', 'firstepisodes','cases'])

    ################################################################################   
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['reinfections', 'firstepisodes','cases']:
            if pd.isna(timeclassificationdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                timeclassificationdf.loc[date, column]=value
            
    timeclassificationdf.fillna(0.0, inplace=True)
    timeclassificationdf.to_pickle("timeclassificationdf.pkl")

    
    return timeclassificationdf

In [7]:
def plot_tcs(gcols, gscale):
    timeclassificationdf[list(gcols)].plot(logy=(gscale=='log'))
    plt.show() 
################################################################################    
series=wdg.SelectMultiple(
    options=['reinfections', 'firstepisodes','cases'],
    value=['reinfections', 'firstepisodes','cases'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale])
#################################################################################
def refresh_graph():
    current=series.value
    if current==(series.options[0],):
        other=(series.options[1],)
    else:
        other=(series.options[0],)
    series.value=other 
    series.value=current 
        
graph=wdg.interactive_output(plot_tcs, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

In [8]:
def access_api(button):
    filters = [
    'areaType=nation', 
    'areaName=England'
    ]
    structure = {
    "date": "date",
    "reinfections": "newReinfectionsBySpecimenDateRollingRate",
    "firstepisodes": "newFirstEpisodesBySpecimenDateRollingRate",
    "cases": "cumCasesByPublishDateRate" 
    }
    api = Cov19API(filters=filters, structure=structure) 
    timeclassification=api.get_json() 

    with open("timeclassification.json", "wt") as OUTF:
        json.dump(timeclassification, OUTF)
    return timeclassification
    
def api_button_callback(button):
    apidata=access_api(button)
    global timeclassificationdf
    timeclassificationdf=wrangle_data(apidata)
    refresh_graph()
    
    apibutton.icon="check"
    apibutton.disabled=True

apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download' 
)

apibutton.on_click(api_button_callback)

display(apibutton)

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

**Author and Copyright Notice**  Sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).